In [20]:
from os.path import dirname, abspath, join
import sys
#import numpy as np
#from sklearn.preprocessing import StandardScaler
import torch 
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from os.path import dirname, abspath, join
import sys

# # Find code directory relative to our directory
# THIS_DIR = dirname(__file__)
# CODE_DIR = abspath(join(THIS_DIR, '..', ''))
sys.path.append('/home/hansalin/Code/Transformer/Test')

type(sys.path)
for path in sys.path:
   print(path)

import os
myhost = os.uname()[1]

print("Host name: ", myhost)


/home/hansalin/Code/Transformer/Test/Code
/home/hansalin/anaconda3/envs/myenv/lib/python38.zip
/home/hansalin/anaconda3/envs/myenv/lib/python3.8
/home/hansalin/anaconda3/envs/myenv/lib/python3.8/lib-dynload

/home/hansalin/.local/lib/python3.8/site-packages
/home/hansalin/anaconda3/envs/myenv/lib/python3.8/site-packages
/home/hansalin/Code/Transformer/Test/dataHandler
/home/hansalin/Code/Transformer/Test
/home/hansalin/Code/Transformer/Test
/home/hansalin/Code/Transformer/Test
Host name:  LAPTOP-9FBI5S57


In [21]:
config = {'model_name': "base_encoder",
            'model':None,
            'embed_size': 64,
            'seq_len': 100,
            'd_model': 512,
            'N': 8,
            'h': 4,
            'dropout': 0.1,
            'num_epochs': 25,
            'batch_size': 32,
            "experiment_name": f"/home/halin/Master/Transformer/Test/ModelsResults/model_{999}/runs",
            "learning_rate": 1e-3,
            "num_parms":0,
            "data_path": '',

          }

In [22]:

import dataHandler.datahandler as dh
import models.models_1 as md
import training.train as tr

In [23]:
trained_model = tr.training(config)


Using device: cpu
Number of paramters: 25204360


FileNotFoundError: [Errno 2] No such file or directory: '/home/halin/Master/Transformer/Test/data/test_data.npy'

: 

In [ ]:
                            
tr.save_data(trained_model=trained_model)


In [ ]:

tr.plot_results(999)